In [ ]:
!pip install --upgrade grpcio grpcio-tools

In [1]:
%matplotlib inline
%load_ext tensorboard

In [2]:
import gymnasium as gym
from gymnasium import spaces
from gymnasium.spaces.utils import flatten
from gymnasium.envs.registration import register, registry
import time
import numpy as np
import pygame

import matplotlib
import matplotlib.pyplot as plt

from typing import Any, Dict
import torch
import torch.nn as nn
import tensorboard

from stable_baselines3 import PPO, A2C
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor

import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from optuna.visualization import plot_optimization_history, plot_param_importances

2025-01-25 09:38:23.413985: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-25 09:38:23.426700: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737797903.442505    1960 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737797903.447563    1960 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-25 09:38:23.464657: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
if 'MarineEnv-v0' not in registry:
    register(
        id='MarineEnv-v0',
        entry_point='environments:MarineEnv',  # String reference to the class
    )

In [ ]:
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
N_TRIALS = 100  # Maximum number of trials
N_JOBS = 1 # Number of jobs to run in parallel
N_STARTUP_TRIALS = 5  # Stop random sampling after N_STARTUP_TRIALS
N_EVALUATIONS = 2  # Number of evaluations during the training
N_TIMESTEPS = int(2e4)  # Training budget
EVAL_FREQ = int(N_TIMESTEPS / N_EVALUATIONS)
N_EVAL_ENVS = 10
N_EVAL_EPISODES = 10
TIMEOUT = int(60 * 15)  # 15 minutes

ENV_ID = 'MarineEnv-v0'

DEFAULT_HYPERPARAMS = {
    "policy": "MlpPolicy",
    "env": ENV_ID,
}

In [ ]:
def sample_ppo_params(trial: optuna.Trial) -> Dict[str, Any]:
    
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1, log=True)  # Learning rate (log scale)
    
    n_steps = 2 ** trial.suggest_int('n_steps', 7, 12)  # Number of steps per update (512-4096)
    
    batch_size = 2 ** trial.suggest_int('batch_size', 5, 10)  # Minibatch size (32-1024)
    
    gamma = trial.suggest_float('gamma', 0.9, 0.9999)  # Discount factor (close to 1 for long-term rewards)
    
    gae_lambda = trial.suggest_float('gae_lambda', 0.8, 1.0)  # GAE lambda (trade-off bias/variance)
    
    clip_range = trial.suggest_float('clip_range', 0.1, 0.3)  # PPO clipping range
    
    ent_coef = trial.suggest_float('ent_coef', 0.0001, 0.1, log=True)  # Entropy coefficient (for exploration)
    
    vf_coef = trial.suggest_float('vf_coef', 0.1, 1.0)  # Value function loss coefficient
    
    max_grad_norm = trial.suggest_float('max_grad_norm', 0.3, 5.0)  # Gradient clipping
    
    target_kl = trial.suggest_float('target_kl', 0.01, 0.2)  # KL divergence target
    
    n_epochs = trial.suggest_int('n_epochs', 3, 10)  # PPO update epochs per batch
    
    activation_fn = trial.suggest_categorical('activation_fn', ['tanh', 'relu'])
    
    net_arch = trial.suggest_categorical('net_arch', ['tiny', 'small'])
    
    # Convert architecture choices
    net_arch = [128, 128] if net_arch == 'tiny' else [256, 256, 256]
    
    activation_fn = {'tanh': nn.Tanh, 'relu': nn.ReLU}[activation_fn]
    
    # Store gamma value in Optuna logs
    trial.set_user_attr('gamma', gamma)

    return {
        'n_steps': n_steps,
        'batch_size': batch_size,
        'gamma': gamma,
        'gae_lambda': gae_lambda,
        'learning_rate': learning_rate,
        'clip_range': clip_range,
        'ent_coef': ent_coef,
        'vf_coef': vf_coef,
        'max_grad_norm': max_grad_norm,
        'target_kl': target_kl,
        'n_epochs': n_epochs,
        'policy_kwargs': {
            'net_arch': net_arch,
            'activation_fn': activation_fn
        }
    }

In [ ]:
class TrialEvalCallback(EvalCallback):
    """
    Callback used for evaluating and reporting a trial.
    
    :param eval_env: Evaluation environement
    :param trial: Optuna trial object
    :param n_eval_episodes: Number of evaluation episodes
    :param eval_freq:   Evaluate the agent every ``eval_freq`` call of the callback.
    :param deterministic: Whether the evaluation should
        use a stochastic or deterministic policy.
    :param verbose:
    """

    def __init__(
        self,
        eval_env: gym.Env,
        trial: optuna.Trial,
        n_eval_episodes: int = 5,
        eval_freq: int = 10000,
        deterministic: bool = True,
        verbose: int = 0,
    ):

        super().__init__(
            eval_env=eval_env,
            n_eval_episodes=n_eval_episodes,
            eval_freq=eval_freq,
            deterministic=deterministic,
            verbose=verbose,
        )
        self.trial = trial
        self.eval_idx = 0
        self.is_pruned = False

    def _on_step(self) -> bool:
        if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
            # Evaluate policy (done in the parent class)
            super()._on_step()
            self.eval_idx += 1
            # Send report to Optuna
            self.trial.report(self.last_mean_reward, self.eval_idx)
            # Prune trial if need
            if self.trial.should_prune():
                self.is_pruned = True
                return False
        return True

In [ ]:
def objective(trial: optuna.Trial) -> float:
    """
    Objective function using by Optuna to evaluate
    one configuration (i.e., one set of hyperparameters).

    Given a trial object, it will sample hyperparameters,
    evaluate it and report the result (mean episodic reward after training)

    :param trial: Optuna trial object
    :return: Mean episodic reward after training
    """

    kwargs = DEFAULT_HYPERPARAMS.copy()

    # 1. Sample hyperparameters and update the keyword arguments
    kwargs.update(**sample_ppo_params(trial))
    print(kwargs)
    # Create the RL model
    model = PPO(device='cpu', verbose=1, **kwargs)
    # Create eval envs
    eval_envs = make_vec_env(ENV_ID, n_envs=N_EVAL_ENVS)

    eval_callback = TrialEvalCallback(eval_envs, trial, N_EVAL_EPISODES, EVAL_FREQ, deterministic=True, verbose=0)

    nan_encountered = False
    try:
        # Train the model
        model.learn(N_TIMESTEPS, callback=eval_callback, progress_bar=True)
    except AssertionError as e:
        # Sometimes, random hyperparams can generate NaN
        print(e)
        nan_encountered = True
    finally:
        # Free memory
        model.env.close()
        eval_envs.close()

    # Tell the optimizer that the trial failed
    if nan_encountered:
        return float("nan")

    if eval_callback.is_pruned:
        raise optuna.exceptions.TrialPruned()

    return eval_callback.last_mean_reward

In [ ]:
# Set pytorch num threads to 1 for faster training
torch.set_num_threads(1)
# Select the sampler, can be random, TPESampler, CMAES, ...
sampler = TPESampler(n_startup_trials=N_STARTUP_TRIALS)
# Do not prune before 1/3 of the max budget is used
pruner = MedianPruner(
    n_startup_trials=N_STARTUP_TRIALS, n_warmup_steps=N_EVALUATIONS // 3
)
# Create the study and start the hyperparameter optimization
study = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")

try:
    study.optimize(objective, n_trials=N_TRIALS, n_jobs=N_JOBS, timeout=TIMEOUT)
except KeyboardInterrupt:
    pass

print("Number of finished trials: ", len(study.trials))

print("Best trial:")
trial = study.best_trial

print(f"  Value: {trial.value}")

print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

print("  User attrs:")
for key, value in trial.user_attrs.items():
    print(f"    {key}: {value}")

# Write report
study.trials_dataframe().to_csv("study_results_ppo_marineenv.csv")

fig1 = plot_optimization_history(study)
fig2 = plot_param_importances(study)

fig1.show()
fig2.show()

In [5]:
env = gym.make('MarineEnv-v0', render_mode='rgb_array', continuous=True, max_episode_steps=400)

In [ ]:
# vec_env = make_vec_env('MarineEnv-v0', n_envs=4)

In [4]:
kwargs = {
    # 'exploration_fraction': 0.7, 
    # 'target_update_interval': 10000, 
    # 'gradient_steps': 3, 
    # 'train_freq': 98, 
    # 'tau': 0.005, 
    # 'batch_size': 128, 
    # 'buffer_size': 10000, 
    'ent_coef': 0.005,
    'gamma': 0.99, 
    'learning_rate': 1e-4, 
    'max_grad_norm': 0.99, 
    'policy_kwargs': {'net_arch': [128, 128], 'activation_fn': torch.nn.ReLU},
}

In [6]:
model = A2C(
    policy='MlpPolicy',
    env=env,
    verbose=1,
    device='cpu', 
    tensorboard_log='./stage_1_tensorboard_logs/',
    **kwargs
)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [7]:
model.learn(total_timesteps=(1e5), progress_bar=True)

Logging to ./stage_1_tensorboard_logs/A2C_1


Output()

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 400      |
|    ep_rew_mean        | 65.6     |
| time/                 |          |
|    fps                | 197      |
|    iterations         | 100      |
|    time_elapsed       | 2        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -2.83    |
|    explained_variance | -16.2    |
|    learning_rate      | 0.0001   |
|    n_updates          | 99       |
|    policy_loss        | -17.3    |
|    std                | 0.998    |
|    value_loss         | 50.3     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 213      |
|    ep_rew_mean        | 93.1     |
| time/                 |          |
|    fps                | 320      |
|    iterations         | 200      |
|    time_elapsed       | 3        |
|    total_timesteps    | 1000     |
|

In [8]:
mean, std = evaluate_policy(model=model, env=env, n_eval_episodes=10, deterministic=True)
print(f'Mean: {mean:.2f}, Std: {std:.2f}')

/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Mean: -281.38, Std: 35.19


In [ ]:
%tensorboard --logdir ./stage_1_tensorboard_logs/ --host=0.0.0.0

In [ ]:
# model.save('stage_1')
# model.load('stage_1')

In [9]:
env = gym.make('MarineEnv-v0', render_mode='human', continuous=True)
state, _ = env.reset()
episode_rewards = 0 
# flatten_state = flatten(env.observation_space, state)
# state = torch.tensor(flatten_state, dtype=torch.float32, device=device).unsqueeze(0)
for _ in range(400):
    action = model.predict(state, deterministic=True)[0] # continuous action space
    action = model.predict(state, deterministic=True)
    # observation, reward, terminated, truncated, info = env.step(action[0]) # discrite action space
    env.render()
    # time.sleep(0.5)
    episode_rewards += reward
    # print('===========================')
    # print(observation)
    if terminated or truncated:
        print(episode_rewards)
        break

env.close()

AttributeError: 'NoneType' object has no attribute 'fill'

In [ ]:
env.close()

In [ ]:
state, _ = env.reset()

In [ ]:
state

In [ ]:
model.predict(state)[0]

In [ ]:
model.batch_size